In [ ]:
# Colab에 Mecab 설치
#!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
#%cd Mecab-ko-for-Google-Colab
#!bash install_mecab-ko_on_colab190912.sh

In [ ]:
# 위의 코드가 작동하지 않을 때 대안 코드 1 실행
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()
#NameError: name 'Tagger' is not defined 오류 발생 시 런타임을 재실행 해주세요.

In [ ]:
# 한글 자모 단위 처리 패키지 설치
!pip install hgtk

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [ ]:
# fasttext 설치
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

fatal: destination path 'fastText' already exists and is not an empty directory.
/content/fastText
make: Nothing to be done for 'opt'.
Processing /content/fastText
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp311-cp311-linux_x86_64.whl size=4508381 sha256=b55368f52aebb698b9f52a16f8363de02a0be09ca1d06f6b8899c1aa76aa5406
  Stored in directory: /tmp/pip-ephem-wheel-cache-s2gl_vxu/wheels/1e/bc/93/e4d717605b2816cf84bf6796e766c05fcc2fd4feb3f170fdf3
Successfully built fasttext
  Attempting uninstall: fasttext
    Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2


## 1. 데이터 로드

In [ ]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk

In [ ]:
# 네이버 쇼핑 리뷰
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x7ee130751750>)

In [ ]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력

전체 리뷰 개수 : 200000


In [ ]:
total_data.head()

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


## 2. hgtk 튜토리얼



word embedding이 단어 단위의 임베딩이었다면, character embedding은 문자 단위의 임베딩입니다. 한국어를 character embedding할 수 있는 것이 바로 자음 모음 분리기 hgtk입니다.

 영어는 하나의 알파벳(52자)를 기준으로 character embedding을 하지만, 한국어에서 하나의 음절별로 character embedding을 하면 11172개의 음절이 있기 때문에 계산량이 너무 많습니다. 따라서 그보다 작은 단위인 자음,모음으로 분리하는 것입니다.

 >참고 repo: https://github.com/bluedisk/hangul-toolkit

In [ ]:
# 한글인지 체크
print(hgtk.checker.is_hangul('ㄱ'))
print(hgtk.checker.is_hangul('12'))
print(hgtk.checker.is_hangul('a'))

True
False
False


In [ ]:
# 음절을 초성, 중성, 종성으로 분해
print(hgtk.letter.decompose('남'))
# 초성, 중성, 종성을 하나의 음절로 결합
print(hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ'))
print(hgtk.letter.decompose('닌'))
print(hgtk.letter.decompose('자'))

('ㄴ', 'ㅏ', 'ㅁ')
남
('ㄴ', 'ㅣ', 'ㄴ')
('ㅈ', 'ㅏ', '')


In [ ]:
# 결합할 수 없는 상황에서는 에러 발생
try:
  hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ') # 중성이 없는 경우
except:
  print('에러 발생')

try:
  hgtk.letter.compose('ㄴ', 'ㅓ','ㅣ')
except:
  print('에러 발생2')

에러 발생
에러 발생2


In [ ]:
hgtk.letter.compose('ㄴ', 'ㅓ')

'너'

## 3. 데이터 전처리

![image.png](attachment:image.png)

In [ ]:
def word_to_jamo(token):
  def to_special_token(jamo): # 경우에 따라 초, 중, 종성이 다 있는 게 아닌 경우도 있다. 이 경우 -를 반환해주는 함수
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)

      # 자모가 빈 문자일 경우 특수문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char

  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

In [ ]:
print(word_to_jamo('남동생'))
print(word_to_jamo('야구')) # 야구의 경우 종성이 없으므로 종성 부분을 -로 반환

ㄴㅏㅁㄷㅗㅇㅅㅐㅇ
ㅇㅑ-ㄱㅜ-


In [ ]:
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


In [ ]:
# mecab으로 형태소를 분리해주고 그 형태소마다 각각 자음모음을 분리해주는 함수
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [ ]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [ ]:
# 리뷰 데이터의 reviews 컬럼만을 가져와서 자모 분리
tokenized_data = []

for sample in tqdm(total_data['reviews'].to_numpy()):
    tokenzied_sample = tokenize_by_jamo(sample) # 자소 단위 토큰화
    tokenized_data.append(tokenzied_sample)

100%|██████████| 200000/200000 [01:00<00:00, 3325.94it/s]


In [ ]:
print(len(tokenized_data))
print("전처리 전:", total_data['reviews'][1])
print("전처리 후:", tokenized_data[1])

200000
전처리 전: 택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
전처리 후: ['ㅌㅐㄱㅂㅐ-', 'ㄱㅏ-', 'ㅇㅓㅇㅁㅏㅇ', 'ㅇㅣ-', 'ㄴㅔ-', 'ㅇㅛㅇ', 'ㅈㅓ-ㅎㅢ-', 'ㅈㅣㅂ', 'ㅁㅣㅌ', 'ㅇㅔ-', 'ㅊㅡㅇ', 'ㅇㅔ-', 'ㅁㅏㄹ', 'ㄷㅗ-', 'ㅇㅓㅄㅇㅣ-', 'ㄴㅘ-ㄷㅜ-', 'ㄱㅗ-', 'ㄱㅏ-', 'ㄱㅗ-']


In [ ]:
tokenized_data[:3]

[['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ'],
 ['ㅌㅐㄱㅂㅐ-',
  'ㄱㅏ-',
  'ㅇㅓㅇㅁㅏㅇ',
  'ㅇㅣ-',
  'ㄴㅔ-',
  'ㅇㅛㅇ',
  'ㅈㅓ-ㅎㅢ-',
  'ㅈㅣㅂ',
  'ㅁㅣㅌ',
  'ㅇㅔ-',
  'ㅊㅡㅇ',
  'ㅇㅔ-',
  'ㅁㅏㄹ',
  'ㄷㅗ-',
  'ㅇㅓㅄㅇㅣ-',
  'ㄴㅘ-ㄷㅜ-',
  'ㄱㅗ-',
  'ㄱㅏ-',
  'ㄱㅗ-'],
 ['ㅇㅏ-ㅈㅜ-',
  'ㅈㅗㅎ',
  'ㅇㅏ-ㅇㅛ-',
  'ㅂㅏ-ㅈㅣ-',
  'ㅈㅓㅇㅁㅏㄹ',
  'ㅈㅗㅎ',
  'ㅇㅏ-ㅅㅓ-',
  '2',
  'ㄱㅐ-',
  'ㄷㅓ-',
  'ㄱㅜ-ㅁㅐ-',
  'ㅎㅐㅆ',
  'ㅇㅓ-ㅇㅛ-',
  'ㅇㅣ-',
  'ㄱㅏ-ㄱㅕㄱ',
  'ㅇㅔ-',
  'ㄷㅐ-ㅂㅏㄱ',
  'ㅇㅣㅂㄴㅣ-ㄷㅏ-',
  '.',
  'ㅂㅏ-ㄴㅡ-ㅈㅣㄹ',
  'ㅇㅣ-',
  'ㅈㅗ-ㄱㅡㅁ',
  'ㅇㅓㅇㅅㅓㅇ',
  'ㅎㅏ-',
  'ㄱㅣㄴ',
  'ㅎㅏ-',
  'ㅈㅣ-ㅁㅏㄴ',
  'ㅍㅕㄴㅎㅏ-',
  'ㄱㅗ-',
  'ㄱㅏ-',
  'ㅅㅓㅇㅂㅣ-',
  'ㅊㅚ-ㄱㅗ-',
  'ㅇㅖ-ㅇㅛ-',
  '.']]

단어를 자모 분리한 것을 역으로 하여 자모 상태를 단어로 다시 결합시키는 함수도 정의해봅시다. 이는 단어의 코사인 유사도를 평가할 때 자모 분리가 된 상태가 아니라 단어 상태로 편리하게 보기 위함입니다.

In [ ]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0

  # 1. 초기 입력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)이 아닐 경우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3

  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']

  word = ''
  try:
    for jamo in tokenized_jamo:

      # 초성, 중성, 종성의 묶음으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종성이 존재하지 않는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 존재하는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo

  # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
  # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
  except Exception as exception:
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 완료
  # word : '남동생'

  return word

## 4. FastText

In [ ]:
import fasttext

fasttext를 실행하기에 앞서 훈련 대상인 단어들을 txt 파일로 준비해둬야 합니다. 따라서 `tokenized_data.txt`라는 파일을 쓰기 모드(w)로 생성해주고 앞서 전처리한 `tokenized_data`를 입력해줍니다.

In [ ]:
with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:00<00:00, 241337.05 line/s]


아래처럼 `train_unsupeviesd` 함수는 훈련을 시켜주는 함수입니다. 인자로 훈련할 단어가 담긴 txt 파일을 지정하고 model을 `cbow`나 `skipgram` 중에 하나를 고르면 됩니다.

In [ ]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [ ]:
model.save_model("fasttext.bin")

In [ ]:
model = fasttext.load_model("fasttext.bin")

In [ ]:
model[word_to_jamo('남동생')] # 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

array([ 2.85749108e-01,  5.32697797e-01,  9.72192764e-01, -1.74561024e-01,
       -9.40615535e-01, -8.52213144e-01,  1.84142113e-01,  7.23624051e-01,
       -6.02583587e-01, -4.90640759e-01,  9.03603256e-01,  2.96650290e-01,
       -2.10492730e-01,  6.45978987e-01, -5.46604753e-01,  6.41264975e-01,
        6.83590710e-01,  5.07914782e-01,  1.90157950e-01, -3.27531621e-02,
        5.80425084e-01, -5.27899086e-01,  6.99647903e-01, -1.41876325e-01,
        5.80996238e-02, -3.13640386e-01,  7.06844479e-02,  1.19922531e+00,
        9.48624730e-01, -7.08683491e-01,  5.12313426e-01, -8.10058236e-01,
        2.31832623e-01, -2.90871948e-01,  1.51137781e+00, -2.15766624e-01,
        4.38416228e-02, -2.49742463e-01,  9.85836610e-02,  1.48401380e-01,
        4.82708663e-01, -9.81113911e-02,  3.92483830e-01, -8.28986242e-02,
       -2.54946172e-01, -1.10600853e+00,  7.52483681e-03,  3.19441855e-01,
       -6.56547397e-02, -2.13221177e-01, -2.11511150e-01, -2.59903312e-01,
        1.69138134e-01,  

`get_nearest_neighbors` 함수를 사용하여 '남동생'이라는 단어와 가장 유사도가 높은 단어들(자모 분리된 상태)을 k개만큼 출력해줍니다.

In [ ]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

[(0.8671373724937439, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.8345811367034912, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7394193410873413, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7316157817840576, 'ㅊㅣㄴㄱㅜ-'),
 (0.7173355221748352, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.7168329358100891, 'ㄴㅏㅁㅇㅏ-'),
 (0.7005258202552795, 'ㅈㅗ-ㅋㅏ-'),
 (0.6888477802276611, 'ㅈㅜㅇㅎㅏㄱㅅㅐㅇ'),
 (0.6667895317077637, 'ㅇㅓㄴㄴㅣ-'),
 (0.6643229126930237, 'ㄴㅏㅁㅈㅏ-')]

앞서 만든 `jamo_to_word`로 가독성이 좋게 출력해봅시다.

In [ ]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))
print(transform(model.get_nearest_neighbors(word_to_jamo('구매'), k=10)))
print(transform(model.get_nearest_neighbors(word_to_jamo('배달'), k=10)))

[('동생', 0.8671373724937439), ('남친', 0.8345811367034912), ('남편', 0.7394193410873413), ('친구', 0.7316157817840576), ('생일', 0.7173355221748352), ('남아', 0.7168329358100891), ('조카', 0.7005258202552795), ('중학생', 0.6888477802276611), ('언니', 0.6667895317077637), ('남자', 0.6643229126930237)]
[('구매처', 0.8485594987869263), ('구입', 0.8151829242706299), ('주문', 0.7182267308235168), ('주문건', 0.6859400868415833), ('구매자', 0.6213594675064087), ('주문서', 0.6170161962509155), ('구메', 0.6099585890769958), ('헤매', 0.5940358638763428), ('구명조끼', 0.5845101475715637), ('구이', 0.5773112177848816)]
[('배송지', 0.796425461769104), ('깨달', 0.7708380818367004), ('매달', 0.7456046342849731), ('운송장', 0.7196446657180786), ('출고가', 0.7136244177818298), ('택배', 0.7071263194084167), ('메달', 0.7013647556304932), ('배소', 0.6943267583847046), ('송장', 0.6902399063110352), ('직송', 0.688640296459198)]


## 5. Word2Vec

![image.png](attachment:image.png)
이제 word2vec를 사용하여 자모 단위로 분리하는 것이 아닌 단어 단위로 분리하여 임베딩 벡터를 생성해볼 것입니다.

In [ ]:
# 간단하게 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

tokenized_data2 = []
for sentence in tqdm(total_data['reviews'].to_list()):
    tokenized_sentence = mecab.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data2.append(stopwords_removed_sentence)

100%|██████████| 200000/200000 [00:22<00:00, 8855.24it/s]


In [ ]:
print("word2vec용 데이터:", tokenized_data2[0])
print("fasttext용 데이터:", tokenized_data[0])

word2vec용 데이터: ['배공', '빠르', '고', '굿']
fasttext용 데이터: ['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']


In [ ]:
#!pip install gensim

In [ ]:
from gensim.models import Word2Vec

model2 = Word2Vec(sentences = tokenized_data2,
                  vector_size = 1000,
                  window = 5,
                  min_count = 5,
                  workers = 4,
                  sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
# 단어의 총 개수는 15005개이고 벡터 차원은 1000으로 축소되었다.
model2.wv.vectors.shape

(15005, 1000)

## 6. FastText와 Word2Vec 결과 비교

### **Q1) 남동생과 주문 두 단어를 input으로 넣고 결과를 비교한 뒤 해석해보세요**

*(Hint: 유사도, 단어의 의미, 단어의 생김새 등을 고려해 볼 수 있습니다)*

In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("남동생"))

FastText 유사도: [('동생', 0.8671373724937439), ('남친', 0.8345811367034912), ('남편', 0.7394193410873413), ('친구', 0.7316157817840576), ('생일', 0.7173355221748352), ('남아', 0.7168329358100891), ('조카', 0.7005258202552795), ('중학생', 0.6888477802276611), ('언니', 0.6667895317077637), ('남자', 0.6643229126930237)]
Word2Vec 유사도: [('사촌', 0.7566232681274414), ('양가', 0.7379902005195618), ('결혼', 0.7374473214149475), ('입학', 0.7257005572319031), ('시어머님', 0.7226743698120117), ('레인', 0.7022974491119385), ('둥이', 0.7011923789978027), ('엄니', 0.701007604598999), ('축하', 0.6974373459815979), ('모임', 0.6929582357406616)]


In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('주문'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("주문"))

FastText 유사도: [('주문건', 0.906598687171936), ('주문서', 0.8494508266448975), ('주문자', 0.7549388408660889), ('주무시', 0.740752637386322), ('주무', 0.7267564535140991), ('구입', 0.7227046489715576), ('구매', 0.718227207660675), ('시켰었', 0.6814103126525879), ('구매처', 0.6687557101249695), ('시켰', 0.6650528907775879)]
Word2Vec 유사도: [('구매', 0.8260665535926819), ('구입', 0.8234701752662659), ('선택', 0.6320903301239014), ('시켰', 0.5545579791069031), ('준비', 0.5493218898773193), ('결제', 0.5459849834442139), ('신청', 0.5352827310562134), ('장만', 0.5310112833976746), ('도착', 0.5190190672874451), ('교체', 0.5060564875602722)]


**Your Inference:** FastText의 결과를 보면 ‘남동생’과 ‘주문’이라는 말의 전체나 일부가 직접적으로 들어간 단어들, 즉 형태적으로 유사한 단어들이 가장 유사한 단어로 나오고 있다. 가령 ‘남동생’을 input으로 넣었을때 ‘동생’, ‘남’친, ‘남’편이, ‘주문’을 input으로 넣었을때 ‘주문’건, ‘주문’서, ‘주문’자 등이 가장 유사한 단어로 나오고 있다. 반면 Word2Vec의 결과는 맥락적으로 관련된 단어들이나 의미적으로 유사한 단어가 가장 유사한 단어로 나온다. 예를 들어 ‘주문’과 비슷한 뜻을 가진 ‘구매’, ‘구입’이 가장 유사 단어로 나오고 있다. 유사도 측면에서는 전반적으로 FastText가 더 높으며, 가장 유사한 단어 기준으로 약 0.1 정도의 차이를 보인다. 결론적으로 Word2Vec은 ‘같이 자주 나오는 단어’나 의미적 유사성 위주로 판단한다면 FastText는 파생어 뿐 아니라 의미적 유사성도 어느정도 반영하는 것으로 보인다.

### **Q2) Fasttext가 Word2Vec보다 항상 성능이 나은지 다양한 input을 넣어서 시도해보세요**

In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('여성'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("여성"))

FastText 유사도: [('남성', 0.7303630709648132), ('자성', 0.7218074798583984), ('어성', 0.7147840857505798), ('배성', 0.7074929475784302), ('점성', 0.7074399590492249), ('성인', 0.7062436938285828), ('내성', 0.6987369060516357), ('지성', 0.698695957660675), ('형성', 0.6943333148956299), ('개성', 0.6798306703567505)]
Word2Vec 유사도: [('남성', 0.8218715786933899), ('유아', 0.6935902237892151), ('성인', 0.691750168800354), ('여자', 0.6887168884277344), ('남자', 0.6706873178482056), ('아동', 0.6660740971565247), ('전문가', 0.6194642782211304), ('77', 0.6020462512969971), ('입문', 0.6013357639312744), ('255', 0.5968115925788879)]


In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('비타민'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("비타민"))

FastText 유사도: [('비타', 0.7778773307800293), ('국민', 0.7022587656974792), ('비비', 0.6605033874511719), ('빋', 0.6560096144676208), ('비교', 0.6530042290687561), ('유목민', 0.6486132144927979), ('유산균', 0.6395082473754883), ('비아', 0.638543426990509), ('비염', 0.635815441608429), ('비늘', 0.6272722482681274)]
Word2Vec 유사도: [('영양제', 0.7844805717468262), ('캐닌', 0.7818204164505005), ('유산균', 0.778425931930542), ('단백질', 0.7715478539466858), ('먹여요', 0.7344641089439392), ('로얄', 0.7281597852706909), ('파스퇴르', 0.7247311472892761), ('쉐이크', 0.7181972861289978), ('영양', 0.7032598853111267), ('즐겨', 0.6893134117126465)]


In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('형광펜'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("형광펜"))

FastText 유사도: [('형광', 0.9588881134986877), ('광색', 0.7967095375061035), ('톤', 0.7679265141487122), ('카키색', 0.7624732851982117), ('미색', 0.760770857334137), ('투톤', 0.7523170709609985), ('블링', 0.751548707485199), ('회색빛', 0.7510120868682861), ('리색', 0.7457737922668457), ('골드색', 0.737829864025116)]
Word2Vec 유사도: [('샴페인', 0.8306828141212463), ('그래이', 0.8287167549133301), ('만수무강', 0.8190242052078247), ('연식', 0.8143661618232727), ('솔리드', 0.8134633302688599), ('집성목', 0.8096138834953308), ('사제', 0.8073872327804565), ('나뭇잎', 0.8061144948005676), ('꾸덕꾸덕', 0.8040460348129272), ('도면', 0.803908109664917)]


In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('좋아'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("좋아"))

FastText 유사도: [('좋아하', 0.9161620140075684), ('좋아해', 0.8811062574386597), ('좋아함', 0.8645984530448914), ('좋아할', 0.8466616272926331), ('짱아', 0.7671179175376892), ('좋아해서', 0.7429694533348083), ('딸아', 0.7383669018745422), ('우아', 0.7154132723808289), ('조카', 0.7142690420150757), ('좋아요', 0.7026201486587524)]
Word2Vec 유사도: [('아라', 0.6587160229682922), ('행복', 0.6260817646980286), ('편해', 0.5964709520339966), ('즐거워', 0.5880709886550903), ('이뻐', 0.5786017775535583), ('깜찍', 0.5693917870521545), ('좋아합니다', 0.5684081315994263), ('조아', 0.5587714910507202), ('신기', 0.5456815958023071), ('뻣', 0.5356332659721375)]


**Your Inference:** FastText가 항상 Word2Vec보다 더 나은 성능을 보이는 것은 아니다. Q1에서 살펴봤듯, FastText는 자모 등 단어 안에 있는 구조를 활용하기 때문에 파생어나 합성어의 유사성을 잘 포착한다. 예를 들어 ‘좋아’라는 단어에 대해서는 ‘좋아하’, ‘좋아해’, ‘좋아함’ 등의 다양한 파생형을 찾아냈다. 반면, ‘여성’이나 ‘비타민’처럼 형태적으로 단순하거나 합성어가 아닌 경우 FastText는 ‘성’이나 ‘비’, ’민’이라는 일부 글자만 똑같은 단어들을 유사한 단어로 판단하고 있어서 직관적으로 유사하다고 보기 어려웠다. (성능 수치가 별로 차이가 안날때 포함) 따라서 상황에 따라 FastText와 Word2Vec의 강점이 다르고 성능 또한 다르다고 판단했다.